In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import json
from pathlib import Path
import pandas as pd
import numpy as np
import random as rnd
import dedupe
import logging
import datetime
from dotenv import find_dotenv, load_dotenv
import dotenv
import requests
from urllib.parse import urljoin
import subprocess

import backtester
from ers_experimenter import ERSExperiment
import utils

log_fmt = f'%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format=log_fmt, datefmt='%Y-%m-%dT%H:%M:%S')
logging.getLogger("requests").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.info('Go head')

load_dotenv(find_dotenv())

INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
INFO:root:Go head


True

In [227]:
def check_to_create_company_entry(company):
    result = False
    matching_ers_ids = []
    
    url = os.environ["ERS_BASEURI"] + '/crud_service/matchCompany'
    response = requests.post(url, json=company)
    if response.status_code == 200:
        matches = response.json()

        if len(matches) == 0:
            result = True
        else:
            matching_ers_ids = matches
        
    elif response.status_code == 500:
        if response.text.find('but maxThreshold is') > 0:
            logging.info(f'For {company} found too ambiguous entries!')
        else:
            logging.warning(f'Got some unkown 500 error: {response.text}. The problem ocurred '
                        f'for record {company}.')
    else:
        logging.warning(f'ERS responded with unexpected status {response.status_code}. The problem ocurred '
                        f'for record {company}.')
        
    return result, matching_ers_ids


def create_company(company):
    url = os.environ["ERS_BASEURI"] + '/crud_service/createCompany'

    response = requests.post(url, json=company)
    if response.status_code != 201:
        logging.warning(f"Couldnt create company entry for {company['crmId']}. ERS replied: {response.text}")

                        
def get_infos_about_company(ers_id):
    url = os.environ["ERS_BASEURI"] + '/crud_service/getCompanyById'
    response = requests.get(url, {'idType':'ERS_ID', 'id':ers_id})
    if response.status_code != 200:
        logging.warning(f'Could not get infos about {ers_id} due to {response.text}')
    else:
        infos = json.loads(response.text)
        if len(infos) == 1:
            return infos[0]
        else:
            logging.warning(f'Returned infos about company {ers_id} was given in unexpected format. Raw response of ERS was: {response.text}')
            return None
        
                    
def compile_infos_about_duplicates(duplicates):
    if len(duplicates) == 0:
        return pd.DataFrame()
                        
    crm_infos_about_duplicates = []
    ers_infos_about_duplicates = []
    for duplicate in duplicates:
        crm_infos_about_duplicates.append({**{'crmId_from_imported_file':duplicate['crmId']}, **crm_data.loc[duplicate['crmId']].reindex(['name','address.street', 'address.city', 'address.zip']).to_dict()})
        base_infos = {'ersId':duplicate['ersId'], 'probability':duplicate['probability']}
        
        ers_infos = get_infos_about_company(duplicate['ersId'])        
        if ers_infos is not None:
            base_infos['ers_name'] = ers_infos.get('name')
            base_infos['crmId_known_to_ERS'] = ers_infos.get('crmId')
            address_infos = ers_infos.get('residentAddressSet')
            if (address_infos is not None) and len(address_infos) >= 1:
                base_infos['ers_street'] = address_infos[0]['address'].get('street')
                base_infos['ers_city'] = address_infos[0]['address'].get('city')
                base_infos['ers_zip'] = address_infos[0]['address'].get('zip')
                
        ers_infos_about_duplicates.append(base_infos)
        
    crm_infos_about_duplicates = pd.DataFrame.from_records(crm_infos_about_duplicates)
    ers_infos_about_duplicates = pd.DataFrame.from_records(ers_infos_about_duplicates)
    result = pd.concat([crm_infos_about_duplicates, ers_infos_about_duplicates], axis=1).fillna('')
    return result[['crmId_from_imported_file', 'crmId_known_to_ERS', 'ersId', 'probability', 'name', 'ers_name', 'address.city', 'ers_city', 'address.street', 'ers_street', 'address.zip', 'ers_zip']]
        

def load_crm_data_into_ers(crm_data):
    experimenter = ERSExperiment()
    
    duplicates = []
    for i, record in enumerate(crm_data.reset_index().to_dict(orient='records')):
        if (i % 1000) == 0:
            logging.info(f'Processing {i}th of {len(crm_data)} entries.')

        try:
            company = experimenter._transform_crm_record_to_company_object(record, information_source=str(crm_filename))

            crm_id = company.pop('crmId')
            is_safe_to_create, matches = check_to_create_company_entry(company)
            if is_safe_to_create:
                company['crmId'] = crm_id
                create_company(company)
            else:
                for match in matches:
                    duplicates.append({'crmId':crm_id, **match})
                logging.warning(f'Found duplicate for CRM Id {crm_id}: {matches}')

        except Exception as e:
            logging.error(f'Exception during calling createCompany: {e}')
                        
    logging.info('Done with loading {len(crm_data)} company entries.')
    return compile_infos_about_duplicates(duplicates)               

# Loading the raw data

In [24]:
ls $DATA_PATH/raw/crefo_cleaning

11761_3.xlsx  csprod_companies__2018_12_05.csv
11761_4.xlsx  csprod_companies__2018_12_05_edited.csv
11761_5.xlsx  ~$11761_5.xlsx


In [3]:
MAP_CRM_TO_ERS_COLUMNS = {'id':'crmId', 'buergel_id':'buergelId', 'name':'name', 'legal_form':'legalForm',
                               'hq_email':'email', 'hq_phone':'phoneNumber', 'website':'website',
                               'taxid':'vatID', 'register_number':'registerNumber', 'local_court':'commercialRegister',
                               'hq_street':'address.street', 'hq_zip_code':'address.postalCode',
                               'hq_city':'address.city', 'hq_country':'address.country'}

crm_filename = Path(os.environ['DATA_PATH']) / 'raw' / 'crefo_cleaning' / 'csprod_companies__2018_12_05_edited.csv'
crm_data = pd.read_csv(crm_filename, skiprows=[44115, 44950, 49610], dtype={'buergel_id':str, 12:str, 24:str, 33:str})

crm_data = crm_data[list(MAP_CRM_TO_ERS_COLUMNS.keys())]
crm_data.rename(columns=MAP_CRM_TO_ERS_COLUMNS, inplace=True)
crm_data.set_index('crmId', inplace=True)
crm_data[:5]

,buergelId,name,legalForm,email,phoneNumber,website,vatID,registerNumber,commercialRegister,address.street,address.postalCode,address.city,address.country
crmId,,,,,,,,,,,,,
1,55013451,Bio River Life Science im Rheinland e.V.,NaN,NaN,NaN,http://www.bioriver.de,NaN,NaN,NaN,Merowinger Platz 1a,40225,Düsseldorf,DE
2,NaN,Die Schuhleister GmbH & Co. KG,NaN,NaN,NaN,http://www.die-schuhleister.de,NaN,NaN,NaN,Eifelplatz 1-3,50677,Köln,DE
3,58088981,evopark GmbH,NaN,NaN,NaN,http://www.evopark.de,NaN,NaN,NaN,Sedanstraße 31-33,50668,Köln,DE
4,3549425,Stadtsparkasse Düsseldorf,NaN,NaN,+492118780,http://www.sskduesseldorf.de,NaN,14082,NaN,Berliner Allee 33,40212,Düsseldorf,DE
5,NaN,Verivox Versicherungsvergleich GmbH,NaN,NaN,NaN,http://www.verivox.de,NaN,NaN,NaN,Am Taubenfeld 10,69123,Heidelberg,DE


In [229]:
# logging.warning('First purging the complete database.')
# _ = requests.post(os.environ["ERS_BASEURI"] + '/aux_service/purgeDatabase')

11:28:32.011 - WARNING - First purging the complete database.


In [230]:
%time duplicates = load_crm_data_into_ers(crm_data[:5])
duplicates.to_pickle(Path(os.environ['DATA_PATH']) / 'processed' / 'crefo_cleaning_duplicates_of_crm.pkl')

11:37:47.011 - INFO - Processing 0th of 5 entries.
11:37:47.011 - INFO - Done with loading {len(crm_data)} company entries.
CPU times: user 30 ms, sys: 30 ms, total: 60 ms
Wall time: 665 ms


In [231]:
duplicates = pd.read_pickle(Path(os.environ['DATA_PATH']) / 'processed' / 'crefo_cleaning_duplicates_of_crm.pkl')

In [232]:
len(duplicates)

15

In [3]:
pwd

'/home/datascientist/host/src/notebooks'

In [19]:
df = pd.read_excel('/home/datascientist/host/data/raw/crefo_cleaning/11761_3.xlsx')

In [7]:
len(df)

44337

In [8]:
df[:3]

,Crefo,Anrede,Titel,Beteiligter Vorname,Beteiligter Nachname,Eigenschaft,Firma1,Firma2,Firma3,Straße/Hausnummer,PLZ,Ort,Telefonnummer,E-Mail,URL,neuester Umsatz,neueste Mitarbeiterzahl,Branchencode,Branche,Bonitätsklasse
0,2010000001,Herr,NaN,Manfred,Otte,Geschäftsführer,Wiener Conditorei,& Caffeehaus Betriebs-GmbH,NaN,Reichsstr. 81,14052,Berlin,+49 30 3641060,info@wiener-conditorei.de,www.wiener-conditorei.de,5575065.63,148.0,1071002,Herstellung von Konditorwaren,1
1,2010000470,Herr,NaN,Norbert,Kunz,Geschäftsführer,Recke Fleischwaren-,Spezialitäten Vertriebs,GmbH & Co. KG,Beusselstr. 44 n-q/G128,10553,Berlin,+49 30 396030,info@recke-fleischwaren.de,www.recke-fleischwaren.de,32800000.00,115.0,46320,Großhandel mit Fleisch und Fleischwaren,1
2,2010000847,Herr,NaN,Meinhard,Mientus,Geschäftsführer,Michael Mientus GmbH,NaN,NaN,Wilmersdorfer Str. 73,10629,Berlin,+49 30 3239077,info@mientus.com,www.mientus.com,15600000.00,60.0,4771002,Einzelhandel mit Herrenbekleidung und Bekleidu...,1


In [20]:
def join_Firma123(row):
    return " ".join([str(row['Firma1']), str(row['Firma2']), str(row['Firma3'])]).strip()

df.loc[:,['Firma1', 'Firma2', 'Firma3']].fillna('', inplace=True)
df['Firma123'] = df.apply(join_Firma123, axis=1)

In [23]:
type(df.loc[0,'Firma3'])

float

In [ ]:
MAP_CREFO_TO_ERS_COLUMNS = {'Firma123':'name', 'legal_form':'legalForm',
                               'E-Mail':'email', 'Telefonnummer':'phoneNumber', 'URL':'website',
                               'Straße/Hausnummer':'address.street', 'PLZ':'address.postalCode',
                               'Ort':'address.city'}

In [5]:
pwd

'/home/datascientist/host/src/notebooks'

# Work directly w/o Neo4J

In [31]:
sys.path.append('/home/datascientist/host/creditshelf-entity-recognition-service/src/python/')

In [32]:
import check_crefo_addresses

13:58:55.017 [Python MainThread] INFO /home/datascientist/host/creditshelf-entity-recognition-service/src/python/entity_matcher.py::dedupe.api ((SimplePredicate: (sameFiveCharStartPredicate, name), SimplePredicate: (sameThreeCharStartPredicate, name)), (SimplePredicate: (commonFourGram, address.street), SimplePredicate: (nearIntegersPredicate, address.postal_code)))
13:58:55.017 [Python MainThread] INFO /home/datascientist/host/creditshelf-entity-recognition-service/src/python/entity_matcher.py::root Loaded dedupe model /home/datascientist/host/src/notebooks/latest.model successfully.


In [46]:
crm_data = check_crefo_addresses.load_crm_data()
crm_data.set_index('crmId', inplace=True)

In [40]:
crefo_filemark = '11761_5'

In [41]:
with open(Path(os.environ['DATA_PATH']) / f'processed/crefo_cleaning_duplicates_of_{crefo_filemark}.json') as f:
    duplicates = json.load(f)
    
len(duplicates)

549

In [67]:
def join_Firma123(row):
        return " ".join([str(row['Firma1']), str(row['Firma2']), str(row['Firma3'])]).strip()

crefo_data = pd.read_excel(f'/home/datascientist/host/data/raw/crefo_cleaning/{crefo_filemark}.xlsx')
crefo_data.set_index('Crefo', inplace=True)
crefo_data.loc[:,['Firma1', 'Firma2', 'Firma3']] = crefo_data.loc[:,['Firma1', 'Firma2', 'Firma3']].fillna('')
crefo_data['Firmename'] = crefo_data.apply(join_Firma123, axis=1)
crefo_data['CRM_Id'] = ''
crefo_data['similarity'] = np.NAN
crefo_data['CRM_name'] = ''
crefo_data['CRM_street'] = ''
crefo_data['CRM_zip'] = ''
crefo_data['CRM_city'] = ''

In [74]:
len(crefo_data)

4202

In [75]:
for duplicate in duplicates:
    if len(duplicate['matches']) > 1:
        logging.info(f"Found multiple matches to CRM for entry {duplicate['record']['ers_id']}")

    crefo_id = int(duplicate['record']['ers_id'][6:])
    crefo_data.loc[crefo_id, 'CRM_Id'] = crm_id = int(duplicate['matches'][0][0])
    crefo_data.loc[crefo_id, 'similarity'] = duplicate['matches'][0][1]

    crefo_data.loc[crefo_id, 'CRM_name'] = crm_data.loc[crm_id, 'name']
    crefo_data.loc[crefo_id, 'CRM_street'] = crm_data.loc[crm_id, 'address.street']
    crefo_data.loc[crefo_id, 'CRM_zip'] = crm_data.loc[crm_id, 'address.postal_code']
    crefo_data.loc[crefo_id, 'CRM_city'] = crm_data.loc[crm_id, 'address.city']

14:19:21.017 [Python MainThread] INFO /home/datascientist/host/creditshelf-entity-recognition-service/src/python/entity_matcher.py::root Found multiple matches to CRM for entry crefo_2151289958
14:19:21.017 [Python MainThread] INFO /home/datascientist/host/creditshelf-entity-recognition-service/src/python/entity_matcher.py::root Found multiple matches to CRM for entry crefo_2151308199
14:19:22.017 [Python MainThread] INFO /home/datascientist/host/creditshelf-entity-recognition-service/src/python/entity_matcher.py::root Found multiple matches to CRM for entry crefo_5190922618
14:19:22.017 [Python MainThread] INFO /home/datascientist/host/creditshelf-entity-recognition-service/src/python/entity_matcher.py::root Found multiple matches to CRM for entry crefo_5230354873
14:19:22.017 [Python MainThread] INFO /home/datascientist/host/creditshelf-entity-recognition-service/src/python/entity_matcher.py::root Found multiple matches to CRM for entry crefo_7290153022
14:19:22.017 [Python MainThrea

In [27]:
crefo_data.reset_index().to_excel(f'/home/datascientist/host/data/processed/{crefo_filemark}_with_matches_to_CRM.xlsx')

In [76]:
duplicates_with_infos = crefo_data[crefo_data.CRM_Id != ''][['CRM_Id', 'similarity', 'Firmename', 'CRM_name', 'Straße/Hausnummer', 'CRM_street', 'PLZ', 'CRM_zip', 'Ort', 'CRM_city']]

In [28]:
print('Done!')

Done!


In [77]:
duplicates_with_infos

,CRM_Id,similarity,Firmename,CRM_name,Straße/Hausnummer,CRM_street,PLZ,CRM_zip,Ort,CRM_city
Crefo,,,,,,,,,,
2010033498,79190,0.793675,TELES Aktiengesellschaft Informationstechnologien,Teles Aktiengesellschaft Informationstechnologien,Ordensmeisterstr. 15-16,Ernst-Reuter-Platz 8,12099,10587,Berlin,Berlin
2010080652,88160,0.820786,Würfel Kunststofftechnik GmbH,Würfel Kunststofftechnik GmbH,Am Jägerberg 7,Am Jägerberg 7,16727,16727,Velten,Velten
2010129387,89202,0.912569,Oberflächentechnik Kläke GmbH,Oberflächentechnik Kläke GmbH,Köpenicker Str. 147,Köpenicker Str. 147,10997,10997,Berlin,Berlin
2010246038,87674,0.936011,Böger Fassaden Bauelemente GmbH,Böger Fassaden Bauelemente GmbH,Dorfstr. 23,Dorfstraße 23,12529,12529,Schönefeld,Schönefeld
2010345289,88670,0.890699,Pneumatik Berlin GmbH PTM,Pneumatik Berlin GmbH PTM,Falkenberger Str. 38-40,Falkenberger Str. 40,13088,13088,Berlin,Berlin
2010458087,90365,0.932553,Linmet Glas-Keramik Handelsgesellschaft mbH,Linmet Glas-Keramik Handelsgesellschaft mbH,Rhinstr. 84,Rhinstr. 86,12681,12681,Berlin,Berlin
2010577693,88156,0.567181,"Basdorf, Lampe & Partner GmbH Formen- und Werk...",Basdorf Lampe & Partner GmbH Formen- und Werkz...,Köpenicker Str. 325 Halle 41,Köpenicker Str. 325,12555,12555,Berlin,Berlin
2010732190,90695,0.814701,SCALA Electronic GmbH,SCALA Electronic GmbH,Ruhlsdorfer Str. 95,Ruhlsdorfer Str. 95,14532,14532,Stahnsdorf,Stahnsdorf
2010796332,87409,0.808390,Epro Tec GmbH,EproTec GmbH,Brebacher Weg 15 Hs 48,Brebacher Weg 15,12683,12683,Berlin,Berlin
